Before running each cell, be sure to change names of files to use and files to be generated, alongside how many cores were used and how m any files are in range.

In [ ]:
import csv
import os

#########################################################
# This script merges all the training data files csv's 
# generated inside each core for each race simulation 
# specified, into one single csv for that particular core. 

#########################################################


# Define the base path and core directories
base_path = "/Users/joesouber/XGBoost_TBBE"
cores = ["Core1", "Core2", "Core3", "Core4", "Core5", "Core6", "Core7", "Core8"]

for core in cores:
    # Generate filenames programmatically for each core
    files = [os.path.join(base_path, core, "TBBE_OD_XGboost", "Application", "getXGBOOstTrainingData_{}.csv".format(i)) for i in range(400)]
    
    # Specify the merged file location for each core
    merged_filename = os.path.join(base_path, core, "TBBE_OD_XGboost", "Application", "merged_result_2_{}.csv".format(core))
    
    with open(merged_filename, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        
        for index, filename in enumerate(files):
            try:
                with open(filename, 'r') as infile:
                    reader = csv.reader(infile)
                    
                    # Skip header only if it's not the first file
                    if index != 0:
                        next(reader, None)
                        
                    # Write rows from current file to the output file
                    writer.writerows(reader)
            except FileNotFoundError:
                print(f"Warning: {filename} not found and will be skipped.")
            except OSError as e:
                print(f"Error: {e} - {filename} could not be processed.")

In [ ]:
import csv
import os

#####################
# This script merges all those merged files above for each core into one single csv file for the entire simulation.
#####################


# Define the base path and the cores
base_path = "/Users/joesouber/XGBoost_TBBE"
cores = ["Core1", "Core2", "Core3", "Core4", "Core5", "Core6", "Core7", "Core8"]

# Generate filenames for the merged core files
merged_files = [os.path.join(base_path, core, "TBBE_OD_XGboost", "Application", "merged_result_2_{}.csv".format(core)) for core in cores]

# Specify the final merged file location
final_merged_filename = os.path.join(base_path, "final_merged_result_2.csv")

with open(final_merged_filename, 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    
    for index, filename in enumerate(merged_files):
        try:
            with open(filename, 'r') as infile:
                reader = csv.reader(infile)
                
                # Skip header only if it's not the first file
                if index != 0:
                    next(reader, None)
                    
                # Write rows from current file to the output file
                writer.writerows(reader)
        except FileNotFoundError:
            print(f"Warning: {filename} not found and will be skipped.")
        except OSError as e:
            print(f"Error: {e} - {filename} could not be processed.")


In [ ]:
import csv
import os

#####################
# This script is to be used when an entirely new simulation has been run and the two functions above have been used. 
# It merges the old combined simulation csv with the newly generated one, so one csv of all simulations is formed.
#####################



# Define the paths to the files to be merged
file1 = "/Users/joesouber/XGBoost_TBBE/final_merged_result_1.csv"
file2 = "/Users/joesouber/XGBoost_TBBE/final_merged_result_2.csv"

# Define the path to the final merged file
final_merged_filename = "/Users/joesouber/XGBoost_TBBE/final_merged_combined.csv"

# Function to merge the files
def merge_files(file_list, output_file):
    with open(output_file, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        
        for index, filename in enumerate(file_list):
            try:
                with open(filename, 'r') as infile:
                    reader = csv.reader(infile)
                    
                    # Skip header only if it's not the first file
                    if index != 0:
                        next(reader, None)
                        
                    # Write rows from current file to the output file
                    writer.writerows(reader)
            except FileNotFoundError:
                print(f"Warning: {filename} not found and will be skipped.")
            except OSError as e:
                print(f"Error: {e} - {filename} could not be processed.")

# Merge the specified files
merge_files([file1, file2], final_merged_filename)

print(f"Merged files into {final_merged_filename}")